In [1]:
import lxml
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
from fake_useragent import UserAgent                
ua = UserAgent()
headers = {'User-Agent': ua.random}


In [3]:
url = "https://www.transfermarkt.es/valencia-cf/kader/verein/1049/plus/0/galerie/0?saison_id=2020"

In [4]:
plantilla21 = requests.get(url, headers=headers)


In [5]:
plantilla21 = BeautifulSoup(plantilla21.text, "html.parser")

In [6]:
temp21={"Nombre":[],
        "Posicion":[],
        "Edad":[],
        "Nacionalidad":[],
        "Valor":[]
        }

SACAR EL VALOR DEL JUGADOR EN EL EQUIPO

In [7]:
for valor in plantilla21.find_all("td", class_ = "rechts hauptlink"):
    temp21["Valor"].append(valor.text)

SACAR NOMBRE JUGADOR

In [8]:
nombres = plantilla21.find_all('img', class_='bilderrahmen-fixed lazy lazy')
for nombre in nombres:
    #jugador24 = nombre["title"]
    temp21["Nombre"].append(nombre["title"])


SACAR EDAD JUGADOR

Veo que desde la posicion 1 y cada 4 saco el TD donde esta la edad ya que ese campo no tiene ningún atributo o clase que pueda diferenciar del resto.
Saco la longitud total de el td para saber su longitud y recorrer cada campo de la edad.

In [9]:
edad = plantilla21.find_all("td",class_ = "zentriert")

longitud=(len(edad)-4) #Saco el total y en el range le pongo 97 porque es el último valor de la cadena que queremos sacar.

valores_edad = [edad[i].text.strip() for i in range(1, longitud, 4)]
temp21["Edad"]=valores_edad

SACAR LA POSICION
Como en el anterior no tenemos una etiqueta ni nada que nos indique el campo de la posicion tenemos que sacarlo
a traves de la posicion de la etiqueta.

In [10]:
posiciones = plantilla21.find_all("table",class_ = "inline-table")

longitud_pos = (len(posiciones))
for i in range(longitud_pos):
    #print(posiciones[i].find_all("td")[2].text.strip())
    temp21["Posicion"].append(posiciones[i].find_all("td")[2].text.strip())

SACAR NACIONALIDAD

saco el td con la clase, veo que los que tienen doble nacionalidad ocupan td diferentes por eso le decimos que coja todos los "img" y la primera posicion de cada td.

In [11]:
nacionalidades = plantilla21.find_all("td", class_='zentriert')
for nacion in nacionalidades:
    img = nacion.find_all("img", class_="flaggenrahmen")
    if img:
        #print(img[0].get("title"))
        temp21["Nacionalidad"].append(img[0].get("title"))

In [12]:
temporada21 =  pd.DataFrame(temp21)

In [13]:
temporada21.head(10)

,Nombre,Posicion,Edad,Nacionalidad,Valor
0,Jasper Cillessen,Portero,32,Países Bajos,"5,00 mill. €"
1,Jaume Doménech,Portero,30,España,"4,00 mill. €"
2,Cristian Rivero,Portero,23,España,300 mil €
3,Unai Etxebarria,Portero,24,España,150 mil €
4,Gabriel Paulista,Defensa central,30,Brasil,"15,00 mill. €"
5,Mouctar Diakhaby,Defensa central,24,Guinea,"10,00 mill. €"
6,Ferro,Defensa central,24,Portugal,"5,00 mill. €"
7,Eliaquim Mangala,Defensa central,30,Francia,"2,50 mill. €"
8,Cristiano Piccini,Defensa central,28,Italia,"2,20 mill. €"
9,Guillem Molina,Defensa central,21,España,600 mil €


In [14]:
temporada21.Edad = temporada21.Edad.astype(int)

In [15]:
temporada21.Valor = temporada21.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\raul_\AppData\Local\Temp\ipykernel_21072\596495582.py:1: SyntaxWarning: invalid escape sequence '\d'
  temporada21.Valor = temporada21.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)


Pasamos los valores a millones, sabemos que los valores altos que habian no corresponden al valor en millones ya que sabemos que 
la plantilla del valencia no tiene jugadores tan valiosos.

In [16]:
temporada21.loc[temporada21.Valor > 100,"Valor"] = temporada21.loc[temporada21.Valor > 100,"Valor"]/1000


In [17]:
temporada21["Valor millones"] = temporada21.Valor 

In [18]:
temporada21 = temporada21.drop(columns=["Valor"])

In [19]:
temporada21

,Nombre,Posicion,Edad,Nacionalidad,Valor millones
0,Jasper Cillessen,Portero,32,Países Bajos,5.00
1,Jaume Doménech,Portero,30,España,4.00
2,Cristian Rivero,Portero,23,España,0.30
3,Unai Etxebarria,Portero,24,España,0.15
4,Gabriel Paulista,Defensa central,30,Brasil,15.00
5,Mouctar Diakhaby,Defensa central,24,Guinea,10.00
6,Ferro,Defensa central,24,Portugal,5.00
7,Eliaquim Mangala,Defensa central,30,Francia,2.50
8,Cristiano Piccini,Defensa central,28,Italia,2.20
9,Guillem Molina,Defensa central,21,España,0.60


In [20]:
#temporada22["Valor millones"].fillna(temporada22["Valor millones"].min, inplace=True)
min_valor = temporada21["Valor millones"].min()
temporada21["Valor millones"].fillna(min_valor, inplace=True)

C:\Users\raul_\AppData\Local\Temp\ipykernel_21072\2711004779.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temporada21["Valor millones"].fillna(min_valor, inplace=True)


In [21]:
ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada21_20.csv"

temporada21.to_csv(ruta, index="False")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\raul_\AppData\Local\Temp\ipykernel_21072\4015276503.py:1: SyntaxWarning: invalid escape sequence '\D'
  ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada21_20.csv"
